### Search am Beispiel Maze

In [113]:
with open ('data/maze01.txt') as f:   # 10 x 30
    lines = f.read().splitlines()
grid = [list(s) for s in lines]

def showGrid():
    for a in grid:
        print(''.join(a))

zeilen, spalten = len(grid), len(grid[0])
showGrid()

###################################################################
#                                                                 #
#                                                                 #
#                                                                 #
#                                                                 #
#                                                                 #
#                                                                 #
#                                                                 #
#                                                                 #
#                                                                 #
#                                                                 #
#             S                            E                      #
#                                                                 #
#                                                                 #
#                                               

In [69]:
def getVonBis():
    for x in range(len(grid)):
        for y in range(len(grid[x])):
            if grid[x][y] == 'S':
                start = (x,y)
            elif grid[x][y] == 'E':
                ende = (x,y)
    return start, ende

startstate, goalstate = getVonBis()
startstate, goalstate

((11, 14), (11, 43))

In [70]:
def goaltest(state):
    return grid[state[0]][state[1]]  == 'E'
goaltest((11,43))

True

In [71]:
def free(state):
    return grid[state[0]][state[1]] != '#'

def nextstates(state):
    tmp = []
    x, y = state
    links, rechts, oben, unten = (x,y-1), (x,y+1), (x-1,y), (x+1,y)
    linksfrei = (y > 0 and free(links))
    rechtsfrei = (y < spalten and free(rechts))
    obenfrei = (x > 0 and free(oben))
    untenfrei = (x < zeilen and free(unten))
    if obenfrei: tmp.append(oben)
    if untenfrei: tmp.append(unten)
    if linksfrei: tmp.append(links)
    if rechtsfrei: tmp.append(rechts)
    return tmp

In [72]:
from collections import deque
def bfs(s):
    ''' 
    s: Startstellung
    returns: Tupel (prev, explored) 
        prev: dictionary mit den Vorgängern der Spielstellungen
            auf dem  Weg zum Ziel, None wenn Ziel nicht gefunden
        explored: set mit den untersuchten states
    '''   
    frontier =  deque([s])
    prev = {s:None}
    explored = set()

    while frontier:
        state = frontier.popleft()
        explored.add(state)
        if goaltest(state):
            return prev, explored
        for v in nextstates(state):
            if v not in prev:
                frontier.append(v)
                prev[v] = state


In [75]:
prev, explored = bfs(startstate)
 

In [76]:
def reconstructPath(prev,state):
    s = state
    tmp = []
    while prev[s] is not None:
        tmp.append(s)
        s = prev[s]
    tmp.reverse()
    return tmp,len(tmp)

path, weglaenge = reconstructPath(prev,goalstate)


In [77]:
def showResult():
    for t in path[:-1]:
        x, y = t
        grid[x][y] = 'o'

    for t in explored:
        x, y = t
        if grid[x][y] == ' ':
            grid[x][y] = '.'

    showGrid()
    print("explored = {}, weglänge = {}".format(len(explored), weglaenge))
    
showResult()

###################################################################
#.................................                                #
#..................................                               #
#...................................                              #
#....................................                             #
#.....................................                            #
#......................................                           #
#.......................................                          #
#........................................                         #
#.........................................                        #
#..........................................                       #
#.............SooooooooooooooooooooooooooooE                      #
#..........................................                       #
#.........................................                        #
#........................................       

#### dfs, greedy, astar

In [109]:
def dfs(s):
    frontier =  [s]
    prev = {s:None}
    explored = set()
    
    while frontier:
        state = frontier.pop()  
        explored.add(state)
        if goaltest(state):
            return prev,explored
        nxt = nextstates(state)
        nxt.reverse()        # die linken Kinder sollen zuletzt auf den frontier-Keller
        for v in nxt:
            if v not in prev:
                frontier.append(v)
                prev[v] = state

def h(state):
    x1, y1 = state
    x2, y2 = goalstate
    return ((x1-x2)**2 + (y1-y2)**2)**0.5
    
from heapq import heappop, heappush
def greedy(s):
    frontier =[(h(s),s)]  
    prev = {s:None}
    explored = set()
    while frontier:
        _ ,state = heappop(frontier)  
        explored.add(state)
        if goaltest(state):
            return prev,explored
        for v in nextstates(state):
            if v not in prev:
                heappush(frontier,(h(v),v))
                prev[v] = state

def astar(s):
    frontier =[(h(s),s)]  
    prev = {s:None}
    explored = set()
    g = {s:0}                         # backword costs 
    while frontier:
        _ ,state = heappop(frontier)  # die Kosten braucht man an der Stelle nicht
        explored.add(state)
        if goaltest(state):
            return prev,explored
        for v in nextstates(state):
            if v not in prev:
                g[v] = g[state]+1
                heappush(frontier,(g[v]+h(v),v))
                prev[v] = state


In [111]:
with open ('data/maze02.txt') as f:   # 10 x 30
    lines = f.read().splitlines()
grid = [list(s) for s in lines]
zeilen, spalten = len(grid), len(grid[0])
startstate, goalstate = getVonBis()

prev, explored = astar(startstate)          # <---- update

path, weglaenge = reconstructPath(prev,goalstate)

#showGrid()
showResult()

###################################################################
#................................................                 #
#.................................................                #
#......................ooooooooooooooooooo.........               #
#......................o#################o........                #
#......................o#   .............o........                #
#......................o#   .............o....#...                #
#......................o#    ...........oo....#...                #
#......................o#    ..........oo.....#..                 #
#......................o#    .oooooooooo.........      #          #
#......................o#     o####################    #          #
#......................o#     ooooooo                  #          #
#......................o#     ......E                  #          #
#......................o################################.         #
#......................o........................